# Query some information from the triples

As a reminder, here are our compentency questions:


| **ID** | **Competency question**                                                  | **Reference** |
|--------|--------------------------------------------------------------------------|---------------|
| 1      | Who is the primary artist involved in the creative process of a song?    | Music Meta    |
| 2      | In what language is written a song?                                      | Core          |
| 3      | Which different languages the lyrics from our Knowledge Graph use?       | Core          |
| 4      | Which are the different sections of a song?                              | Music Lyrics  |
| 5      | What are the lyrics associated to an specific section of a song?         | Music Lyrics  |
| 6      | What are the starting and ending time stamps of a specific lyric's line? | Music Lyrics  |
| 7      | What is the duration of a lyric's line?                                  | Music Lyrics  |
| 8      | What is the duration of a song's section?                                | Music Lyrics  |
| 9      | What are the singers in charge of singing a specific song's section?     | Music Lyrics  |

Let's try to answer them by performing some SPARQL queries in our knowledge graph's triples.

In [1]:
import os
from rdflib import Graph

In [2]:
# Directory containing the RDF files
triples_folder = './triples/'

# Load all RDF files into a single graph
graph = Graph()
for file_name in os.listdir(triples_folder):
    if file_name.endswith('.ttl') or file_name.endswith('.rdf'):
        graph.parse(os.path.join(triples_folder, file_name), format='turtle')

In [4]:
# SPARQL Queries
queries = {
    "primary_artist": """
        SELECT ?song ?artist WHERE {
            ?song a <https://w3id.org/polifonia/ontology/music-meta/1.0/MusicEntity> ;
                  <http://vivoweb.org/ontology/core#hasTitle> ?title ;
                  <https://w3id.org/polifonia/ontology/music-meta/1.0/isCreatedBy> ?creativeProcess .
            ?creativeProcess <http://vivoweb.org/ontology/core#involvesAgent> ?artistURI .
            ?artistURI <http://vivoweb.org/ontology/core#name> ?artist .
        }
    """,
    "song_language": """
        SELECT ?song ?language WHERE {
            ?song a <https://w3id.org/polifonia/ontology/music-meta/1.0/MusicEntity> ;
                  <http://vivoweb.org/ontology/core#hasLanguage> ?language .
        }
    """,
    "different_languages": """
        SELECT DISTINCT ?language WHERE {
            ?song a <https://w3id.org/polifonia/ontology/music-meta/1.0/MusicEntity> ;
                  <http://vivoweb.org/ontology/core#hasLanguage> ?language .
        }
    """,
    "song_sections": """
        SELECT DISTINCT ?section WHERE {
            ?paragraph a ?section .
        }
    """,
    "lyrics_associated_to_section": """
        SELECT ?section ?lyrics WHERE {
            ?paragraph a ?section ;
                       <https://w3id.org/polifonia/ontology/music-lyrics/1.0/hasTextPart> ?lyricsURI .
            ?lyricsURI <https://w3id.org/polifonia/ontology/music-meta/1.0/hasText> ?lyrics .
        }
    """,
    "line_time_stamps": """
        SELECT ?line ?start ?end WHERE {
            ?lineURI a <https://w3id.org/polifonia/ontology/music-lyrics/1.0/TextFragment> ;
                     <https://w3id.org/polifonia/ontology/music-lyrics/1.0/hasMusicTimeInterval> ?intervalURI .
            ?intervalURI <http://vivoweb.org/ontology/core#hasStartMusicTimeIndex> ?startURI ;
                        <http://vivoweb.org/ontology/core#hasEndMusicTimeIndex> ?endURI .
            ?startURI <http://vivoweb.org/ontology/core#hasMusicTimeIndexComponent> ?start .
            ?endURI <http://vivoweb.org/ontology/core#hasMusicTimeIndexComponent> ?end .
        }
    """,
    "line_duration": """
        SELECT ?line ?duration WHERE {
            ?lineURI a <https://w3id.org/polifonia/ontology/music-lyrics/1.0/TextFragment> ;
                     <https://w3id.org/polifonia/ontology/music-lyrics/1.0/hasMusicTimeInterval> ?intervalURI .
            ?intervalURI <http://vivoweb.org/ontology/core#hasMusicTimeDuration> ?durationURI .
            ?durationURI <http://vivoweb.org/ontology/core#hasValue> ?duration .
        }
    """,
    "section_duration": """
        SELECT ?section ?duration WHERE {
            ?sectionURI a <https://w3id.org/polifonia/ontology/music-meta/1.0/AbstractScore> ;
                        <http://vivoweb.org/ontology/core#hasPart> ?part .
            ?part <http://vivoweb.org/ontology/core#hasMusicTimeDuration> ?durationURI .
            ?durationURI <http://vivoweb.org/ontology/core#hasValue> ?duration .
        }
    """,
    "singers_for_section": """
        SELECT ?section ?singer WHERE {
            ?sectionURI a <https://w3id.org/polifonia/ontology/music-meta/1.0/AbstractScore> ;
                        <https://w3id.org/polifonia/ontology/music-lyrics/1.0/sungBy> ?singerURI .
            ?singerURI <http://vivoweb.org/ontology/core#name> ?singer .
        }
    """
}

In [5]:
# Execute queries and print results
for question, query in queries.items():
    print(f"\nResults for: {question.replace('_', ' ').title()}\n")
    results = graph.query(query)
    for row in results:
        print(row)


Results for: Primary Artist

(rdflib.term.BNode('n7ebf8dc448eb46fea5b69ba4bb9595f3b1'), rdflib.term.Literal('Barbra Streisand'))
(rdflib.term.BNode('n7df3415199da45ea911f2e2cb1a32eeab1'), rdflib.term.Literal('Europe'))
(rdflib.term.BNode('nb8cf2a8cd1854a0d856ed73e9448b0c9b1'), rdflib.term.Literal('Roger Cicero'))
(rdflib.term.BNode('n65d9f16303664c5fa6053b0dec6b398db1'), rdflib.term.Literal('Five For Fighting'))
(rdflib.term.BNode('n7831802ce1784794b62e6009390881b4b1'), rdflib.term.Literal('Tool'))
(rdflib.term.BNode('n847e40c9f6614e3f87ef516c42e0f613b1'), rdflib.term.Literal('Lotto King Karl'))
(rdflib.term.BNode('n02df310df7d64445a9a653d30324623bb1'), rdflib.term.Literal('Deftones'))
(rdflib.term.BNode('nea71578c34064c1087e7edb0b7af3f39b1'), rdflib.term.Literal('The Black Mages'))
(rdflib.term.BNode('n41fd1c3c3e784e28a30cc2a994bb8a59b1'), rdflib.term.Literal('Bloodhound Gang'))
(rdflib.term.BNode('n97b648c5171847c69fbf182066ca6fc5b1'), rdflib.term.Literal('Bon Jovi'))
(rdflib.term.B